In [5]:
from data._utils import load_Leifer2023
Leifer2023 = load_Leifer2023()

In [7]:
Leifer2023['worm0'].keys()

dict_keys(['dataset', 'worm', 'calcium_data', 'smooth_calcium_data', 'residual_calcium', 'smooth_residual_calcium', 'all_neuron_to_idx', 'idx_to_all_neuron', 'max_time', 'time_in_seconds', 'dt', 'num_all_neurons', 'num_named_neurons', 'num_unknown_neurons', 'named_neurons_mask', 'unknown_neurons_mask', 'neurons_mask', 'named_neuron_to_idx', 'idx_to_named_neuron', 'unknown_neuron_to_idx', 'idx_to_unknown_neuron', 'slot_to_named_neuron', 'named_neuron_to_slot', 'slot_to_unknown_neuron', 'unknown_neuron_to_slot', 'slot_to_neuron', 'neuron_to_slot'])